<a href="https://colab.research.google.com/github/AlessioLampis/AlessioLampis/blob/master/Copy_of_Sound_classification_by_Alessio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###This fist part connects the google drive account and loads the urbansound8k data in google collab

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [2]:
from google.colab import drive

drive.mount('/content/gdrive')
root_path = 'gdrive/My Drive/'  #change dir to your project folder

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


# Sound classification

Data Analysis

In [0]:
#import all necessary libraries
import os
import pandas as pd
import IPython as IP
import struct
import matplotlib.pyplot as plt
import numpy as np
import librosa
#import helpers
#from include import helpers

In [5]:
#set the path to the original dataset
us8k_path = 'gdrive/My Drive/urbansound8k'
print(us8k_path)
# Global settings
metadata_path = us8k_path + '/UrbanSound8K.csv'
print(metadata_path)


gdrive/My Drive/urbansound8k
gdrive/My Drive/urbansound8k/UrbanSound8K.csv


In [6]:
# Load metadata as a Pandas dataframe
metadata = pd.read_csv(metadata_path)

# Examine dataframe's head
metadata.head()

,slice_file_name,fsID,start,end,salience,fold,classID,class
0,100032-3-0-0.wav,100032,0.0,0.317551,1,5,3,dog_bark
1,100263-2-0-117.wav,100263,58.5,62.500000,1,5,2,children_playing
2,100263-2-0-121.wav,100263,60.5,64.500000,1,5,2,children_playing
3,100263-2-0-126.wav,100263,63.0,67.000000,1,5,2,children_playing
4,100263-2-0-137.wav,100263,68.5,72.500000,1,5,2,children_playing


In [7]:
# Class distribution
metadata['class'].value_counts()

jackhammer          1000
drilling            1000
air_conditioner     1000
engine_idling       1000
street_music        1000
dog_bark            1000
children_playing    1000
siren                929
car_horn             429
gun_shot             374
Name: class, dtype: int64

In [0]:
def extract_features(file_name):

        X, sample_rate = librosa.load(file_name, res_type='kaiser_fast') 
        mfccs = librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40)
        mfccs_scaled = np.mean(mfccs.T,axis=0)
       	return mfccs_scaled


features = []

# Iterate through each sound file and extract the features 
for index, row in metadata.iterrows():
    
    file_name = us8k_path + '/' + 'fold'+ str(row["fold"]) + '/' + str(row["slice_file_name"])
    class_label = row["class"]
    fold = row["fold"]
    data = extract_features(file_name)
    
    features.append([data, class_label,fold])

# Convert into a Panda dataframe 
featuresdf = pd.DataFrame(features, columns=['feature','class_label','fold'])

Example of on feature

Explaination of librosa mfcc extraction

In [10]:
print('Finished feature extraction from ', len(featuresdf), ' files')
y, sr = librosa.load(us8k_path + '/' + 'fold1'+ '/' + '101415-3-0-2.wav', res_type='kaiser_fast')
mfcc = librosa.feature.mfcc(y, sr, n_mfcc=40)
mfccs_scaled = np.mean(mfccs.T,axis=0)
plt.figure(figsize=(12, 4))
librosa.display.specshow(mfcc_scaled,x_axis='time', y_axis='mel')
plt.colorbar()
plt.title('MFCC')
plt.tight_layout()
plt.show()

Finished feature extraction from  8732  files


NameError: ignored

In [20]:
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical

# Convert features and corresponding classification labels into numpy arrays
X = np.array(featuresdf.feature.tolist())
y = np.array(featuresdf.class_label.tolist())

# Encode the classification labels
le = LabelEncoder()
yy = to_categorical(le.fit_transform(y)) 

# split the dataset 
from sklearn.model_selection import train_test_split 

x_train, x_test, y_train, y_test = train_test_split(X, yy, test_size=0.2, random_state = 42)


[[0. 0. 0. ... 0. 1. 0.]
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 1. 0.]]


In [0]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D
from keras.optimizers import Adam
from keras.utils import np_utils
from sklearn import metrics 

num_labels = yy.shape[1]
filter_size = 2

# Construct model 
model = Sequential()

model.add(Dense(256, input_shape=(40,)))
model.add(Activation('relu'))
model.add(Dropout(0.5))

model.add(Dense(256))
model.add(Activation('relu'))
model.add(Dropout(0.5))

model.add(Dense(num_labels))
model.add(Activation('softmax'))

In [0]:

# Compile the model
model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam')

In [23]:

# Display model architecture summary 
model.summary()

# Calculate pre-training accuracy 
score = model.evaluate(x_test, y_test, verbose=0)
accuracy = 100*score[1]

print("Pre-training accuracy: %.4f%%" % accuracy)

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 256)               10496     
_________________________________________________________________
activation_1 (Activation)    (None, 256)               0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 256)               65792     
_________________________________________________________________
activation_2 (Activation)    (None, 256)               0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 10)               

In [26]:

from keras.callbacks import ModelCheckpoint 
from datetime import datetime 

num_epochs = 100
num_batch_size = 32

checkpointer = ModelCheckpoint(filepath='saved_models/weights.best.basic_mlp.hdf5', 
                               verbose=1, save_best_only=True)
start = datetime.now()

model.fit(x_train, y_train, batch_size=num_batch_size, epochs=num_epochs, validation_data=(x_test, y_test), callbacks=[checkpointer], verbose=1)


duration = datetime.now() - start
print("Training completed in time: ", duration)

Train on 6985 samples, validate on 1747 samples
Epoch 1/100
6985/6985 [==============================] - 1s 80us/step - loss: 2.0935 - accuracy: 0.2763 - val_loss: 1.9455 - val_accuracy: 0.3440

Epoch 00001: val_loss improved from inf to 1.94553, saving model to saved_models/weights.best.basic_mlp.hdf5
Epoch 2/100
6985/6985 [==============================] - 1s 80us/step - loss: 1.9384 - accuracy: 0.3194 - val_loss: 1.7383 - val_accuracy: 0.4379

Epoch 00002: val_loss improved from 1.94553 to 1.73827, saving model to saved_models/weights.best.basic_mlp.hdf5
Epoch 3/100
6985/6985 [==============================] - 1s 78us/step - loss: 1.8149 - accuracy: 0.3516 - val_loss: 1.6516 - val_accuracy: 0.4436

Epoch 00003: val_loss improved from 1.73827 to 1.65163, saving model to saved_models/weights.best.basic_mlp.hdf5
Epoch 4/100
6985/6985 [==============================] - 1s 80us/step - loss: 1.7031 - accuracy: 0.3950 - val_loss: 1.4940 - val_accuracy: 0.4986

Epoch 00004: val_loss improve

In [27]:
# Evaluating the model on the training and testing set
score = model.evaluate(x_train, y_train, verbose=0)
print("Training Accuracy: ", score[1])

score = model.evaluate(x_test, y_test, verbose=0)
print("Testing Accuracy: ", score[1])

Training Accuracy:  0.9331424236297607
Testing Accuracy:  0.8763594627380371
